In [1]:
#start
import gspread #type:ignore NOQA
import imaplib
import email
import json

#connecting to GSeets and selecting the file
sa = gspread.service_account(filename="sa_creds.json")

#read credentials from json
with open("ym_creds.json", "r") as f:
    jmCreds = json.load(f)

In [2]:
maximaEmailList = [] #list to write to GSheets
def maximaEmails(emailList):
    decodedemailList = emailList[0].decode()
    singleEmails = decodedemailList.split()
    for i in singleEmails:
        f = ['Maxima']
        f.append(i)
        maximaEmailList.append(f)

In [3]:
sh = sa.open("Grocery")
#selecting sheets:
wksEmails = sh.worksheet("Emails")

In [4]:
wksEmails.get()
maximaAlreadyIn = [i[1] for i in wksEmails.get() if 'Maxima' in i[0]]

In [5]:
# Connect to the Yahoo Mail IMAP server
imap_server = imaplib.IMAP4_SSL("imap.mail.yahoo.com")
# Login to the account
imap_server.login(jmCreds["email"], jmCreds["password"])
# Select the "Inbox" folder
imap_server.select("Inbox")

('OK', [b'5528'])

In [6]:
# Search for all emails: status, email_ids = imap_server.search(None, "ALL")
# Search for all emails from MAXIMA
status, email_ids = imap_server.search(None, "FROM noreply.code.provider@maxima.lt")
maximaEmails = email_ids[0].decode().split()

In [7]:
maximaToAdd = [i for i in maximaEmails if i not in maximaAlreadyIn]
maximaToAdd

['5468', '5482', '5495', '5497']

In [8]:
for i in maximaToAdd:
    status, email_data = imap_server.fetch(i, "(RFC822)")
    email_message = email.message_from_bytes(email_data[0][1])

In [9]:
#get message bu ID
status, email_data = imap_server.fetch("5497", "(RFC822)")
email_message = email.message_from_bytes(email_data[0][1])

In [10]:
msg1 = email_message.get_payload()[0]
msg1body = msg1.get_payload(decode=True)
msg2 = email_message.get_payload()[0]
msg2body = msg2.get_payload(decode=True)
msg3 = email_message.get_payload()[0]
msg3body = msg3.get_payload(decode=True)

In [11]:
# print(msg1.get_payload(decode=True), msg2.get_payload(decode=True))
print(msg2body.decode())

<html><body>


<html>

<body>

    <div class="previewText" style="display:none; max-height: 0px; overflow: hidden">
            <p>Apsipirkimo suma: 13.48 EUR</p>
            <br />
                <p>Kvito nuolaidų suma: 3.08 EUR</p>
                <br />
                <p> MAXIMOS PINIGŲ likutis: 0.13 EUR </p>
            <p>
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;

In [9]:

# Check if the email is a multipart email
if email_message.is_multipart():
    # Loop through the message parts
    for part in email_message.get_payload():
        # Decode the body of each part
        decoded_body = part.get_payload(decode=True)
        # Print the decoded body
        print(decoded_body.decode())
else:
    # Decode the body of the email
    decoded_body = email_message.get_payload(decode=True)
    # Print the decoded body
    print(decoded_body.decode())

<html><body>


<html>

<body>

    <div class="previewText" style="display:none; max-height: 0px; overflow: hidden">
            <p>Apsipirkimo suma: 13.48 EUR</p>
            <br />
                <p>Kvito nuolaidų suma: 3.08 EUR</p>
                <br />
                <p> MAXIMOS PINIGŲ likutis: 0.13 EUR </p>
            <p>
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfe in position 2: invalid start byte

In [32]:
# Disconnect from the server
imap_server.close()
imap_server.logout()

('BYE', [b'IMAP4rev1 Server logging out'])